In [ ]:
import dash
import pandas as pd
import numpy as np
import dash_table
import logging
import plotly.graph_objs as go
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from optimize_price import *
from optimize_quantity import *
import dash_daq as daq
from dash import dcc

group_colors = {"control": "light blue", "reference": "red"}

app = dash.Dash(
    __name__, meta_tags=[
        {"name": "viewport", "content": "width=device-width"}],
)

server = app.server

# Load the data
sold = pd.read_csv('/Users/kitanolowofoyeku/Cafe+-+Sell+Meta+Data.csv')
transactions = pd.read_csv('/Users/kitanolowofoyeku/Cafe+-+Transaction+-+Store.csv')
date_info = pd.read_csv('/Users/kitanolowofoyeku/Cafe+-+DateInfo.csv')
date_info['HOLIDAY'] = date_info['HOLIDAY'].fillna("Nonholiday")
data1 = pd.merge(sold.drop(['ITEM_ID'],axis=1), transactions.drop(['SELL_CATEGORY'], axis= 1), on =  'SELL_ID')
b = data1.groupby(['SELL_ID', 'SELL_CATEGORY', 'ITEM_NAME', 'CALENDAR_DATE','PRICE']).QUANTITY.sum()
intermediate_data = b.reset_index()
combined_data = pd.merge(intermediate_data, date_info, on = 'CALENDAR_DATE')
combined_data['HOLIDAY'] = combined_data['HOLIDAY'].replace(['Luner New Year','Labor Day','Dragon Boat Festivel','Mid-Autumn Day'],'Holiday')
bau_data = combined_data[(combined_data['IS_SCHOOLBREAK']==0) & (combined_data['IS_WEEKEND']==0)]
product_list = bau_data['SELL_ID'].unique().tolist()
holiday_list = bau_data['HOLIDAY'].unique().tolist()
# App Layout
app.layout = html.Div(
    children=[
        # Error Message
        html.Div(id="error-message"),
        # Top Banner
        html.Div(
            className="study-browser-banner row",
            children=[
                html.H2(className="h2-title",
                        children="PRODUCT PRICE OPTIMIZATION"),

                

            ],
        ),

        html.Div(
            [
                html.Div(
                    [
                        html.Div(
                            className="padding-top-bot",
                            children=[
                                html.H6("By Kitan Olowofoyeku"),
                                dcc.RadioItems(
                                    id="selected-var-opt",
                                    options=[
                                        {
                                            #"label": "Price",
                                            #"value": "price"
                                        },
                                        {
                                            #"label": "Quantity",
                                            #"value": "quantity"
                                        },

                                    ],
                                    value="price",
                                    labelStyle={
                                        "display": "inline-block",
                                        "padding": "12px 12px 12px 12px",
                                    },
                                ),
                            ],
                        ),
                        html.Br(),
                        html.Div(
                            className="padding-top-bot",
                            children=[

                                html.H6("OPTIMIZATION RANGE"),
                                html.Div(
                                    id='output-container-range-slider'),
                                dcc.RangeSlider(
                                    id='my-range-slider',
                                    min=0,
                                    max=500,
                                    step=1,
                                    marks={
                                        0: '0',
                                        500: '500'
                                    },
                                    value=[0, 30]
                                ),



                            ],
                        ),
                         html.Br(),
                        html.Div(
                            className="padding-top-bot",
                            children=[
                                html.H6("Nonholiday/Holiday"),
                                dcc.Dropdown(holiday_list,
                                    id='holiday-opt', 
                                             value='Nonholiday'
                                    
                                )
                                # dcc.Input(
                                #     id = "selected-cost-opt",
                                #     placeholder='Enter fixed cost...',
                                #     type='text',
                                #     value='80'
                                # ),

                            ],
                        ),
                        
                         html.Br(),
                        html.Div(
                            className="padding-top-bot",
                            children=[
                                html.H6("PRODUCT ID"),
                                dcc.Dropdown(product_list,
                                    id='product-opt', 
                                             value=1070
                                    
                                )
                                # dcc.Input(
                                #     id = "selected-cost-opt",
                                #     placeholder='Enter fixed cost...',
                                #     type='text',
                                #     value='80'
                                # ),

                            ],
                        ),


                        html.Br(),
                        html.Div(
                            className="padding-top-bot",
                            children=[
                                html.H6("TOTAL COST"),
                                daq.NumericInput(
                                    id='selected-cost-opt',
                                    min=0,
                                    max=10000,
                                    value=5
                                )
                                # dcc.Input(
                                #     id = "selected-cost-opt",
                                #     placeholder='Enter fixed cost...',
                                #     type='text',
                                #     value='80'
                                # ),

                            ],
                        ),
                        html.Br(),
                        html.Br(),
                        html.Br(),
                        
                        html.H6("RECOMMENDATION:"),
                        html.Div(
                            id = 'id-insights', style={'color': 'DarkCyan', 'fontSize': 15} 
                        ),
                        


                    ],
                    className="pretty_container two columns",
                    id="cross-filter-options",
                ),

            ],
        ),


        html.Div(
            [
                html.Div(
                    [
                        html.Div(
                            className="twelve columns card-left",
                            children=[
                                html.Div(
                                    className="padding-top-bot",
                                    children=[
                                        html.H6("PRICE VS QUANTITY"),
                                        dcc.Graph(id="lineChart2"),
                                    ],
                                )
                            ],
                        ),


                    ],
                    className="pretty_container four columns",

                ),
                html.Div(
                    [
                        html.Div(
                            className="twelve columns card-left",
                            children=[
                                html.Div(
                                    className="padding-top-bot",
                                    children=[
                                        html.H6("MAXIMIZING PROFIT"),
                                        dcc.Graph(id="lineChart1"),
                                    ],
                                )
                            ],
                        ),


                    ],
                    className="pretty_container four columns",

                ),


                html.Div(
                    [
                        html.Div(
                            className="twelve columns card-left",
                            children=[
                                html.Div(
                                    className="padding-top-bot",
                                    children=[
                                        # html.Div(id="table1"),
                                        html.H6("SIMULATED RESULT"),
                                        dash_table.DataTable(

                                            id='heatmap',
                                            columns=[
                                                {'name': 'Profit', 'id': 'Revenue',
                                                    'type': 'numeric'},
                                                {'name': 'Price', 'id': 'Price',
                                                    'type': 'numeric'},
                                                {'name': 'Quantity', 'id': 'Quantity',
                                                    'type': 'numeric'},
                                            ],
                                            style_data_conditional=[
                                                {
                                                    'if': {'row_index': 'odd'},
                                                    'backgroundColor': 'rgb(248, 248, 248)'
                                                },
                                                {
                                                    'if': {
                                                        'row_index': 0,  # number | 'odd' | 'even'
                                                        'column_id': 'Revenue'
                                                    },
                                                    'backgroundColor': 'dodgerblue',
                                                    'color': 'white'
                                                },
                                                {
                                                    'if': {
                                                        'row_index': 0,  # number | 'odd' | 'even'
                                                        'column_id': 'Price'
                                                    },
                                                    'backgroundColor': 'dodgerblue',
                                                    'color': 'white'
                                                },
                                                {
                                                    'if': {
                                                        'row_index': 0,  # number | 'odd' | 'even'
                                                        'column_id': 'Quantity'
                                                    },
                                                    'backgroundColor': 'dodgerblue',
                                                    'color': 'white'
                                                },
                                            ],
                                            style_header={
                                                'backgroundColor': 'rgb(230, 230, 230)',
                                                'fontWeight': 'bold',
                                                # 'border': '1px solid black'
                                            },
                                            style_data={
                                                'whiteSpace': 'normal',
                                                'height': 'auto',
                                            },
                                            editable=True,
                                            filter_action="native",
                                            sort_action="native",
                                            page_size=10,

                                        ),
                                    ],
                                )
                            ],
                        ),

                    ],
                    className="pretty_container two columns",

                ),
            ],
        ),
    ]
)


@app.callback(
    dash.dependencies.Output('output-container-range-slider', 'children'),
    [dash.dependencies.Input('my-range-slider', 'value')])
def update_output(value):
    return "{}".format(value)


@app.callback(
    [
        Output("heatmap", 'data'),
        Output("lineChart1", 'figure'),
        Output("lineChart2", 'figure'),
        Output("id-insights", 'children'), 
    ],
    [
        Input("selected-var-opt", "value"),
        Input("my-range-slider", "value"),
        Input("selected-cost-opt", "value"),
        Input("product-opt", "value"),
        Input("holiday-opt", "value")
    ]
)
def update_output_All(var_opt, var_range, var_cost, product, holiday):

    try:
        if var_opt == 'price':
            res, fig_PriceVsRevenue, fig_PriceVsQuantity, opt_Price, opt_Revenue = fun_optimize(
                var_opt, var_range, var_cost, bau_data, product, holiday)
            res = np.round(res.sort_values(
                'Revenue', ascending=False), decimals=2)

            if opt_Revenue > 0:
                return [res.to_dict('records'), fig_PriceVsRevenue, fig_PriceVsQuantity, 
                    f'The maximum profit of {opt_Revenue} is achieved by optimizing {var_opt} of {opt_Price}, total cost of {var_cost} and optimization was carried for {var_opt} range between {var_range}']
            else:
                return [res.to_dict('records'), fig_PriceVsRevenue, fig_PriceVsQuantity, 
                    f'For the fixed cost of {var_cost} and {var_opt} range between {var_range}, you will incur loss in revenue']

        else:
            res, fig_QuantityVsRevenue, fig_PriceVsQuantity, opt_Quantity,opt_Revenue  = fun_optimize_q(
                var_opt, var_range, var_cost, bau_data, product, holiday)
            res = np.round(res.sort_values(
                'Revenue', ascending=False), decimals=2)
            

            if opt_Revenue  > 0:
                return [res.to_dict('records'), fig_QuantityVsRevenue, fig_PriceVsQuantity, 
                    f'The maximum profit per unit of {opt_Revenue} is achieved by optimizing {var_opt} of {opt_Quantity}, fixed cost of {var_cost} and optimization was carried for {var_opt} range between {var_range}']
            else:
                return [res.to_dict('records'), fig_QuantityVsRevenue, fig_PriceVsQuantity, 
                    f'For the fixed cost of {var_cost} and {var_opt} range between {var_range}, you will incur a loss']
    
    
    except Exception as e:
        logging.exception('Something went wrong with interaction logic:', e)


if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False, dev_tools_ui=False)

/var/folders/5c/49s8xx0s0r32bkg_t49vfkym0000gn/T/ipykernel_4566/2595524309.py:4: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table
/var/folders/5c/49s8xx0s0r32bkg_t49vfkym0000gn/T/ipykernel_4566/2595524309.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/5c/49s8xx0s0r32bkg_t49vfkym0000gn/T/ipykernel_4566/2595524309.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/kitanolowofoyeku/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tools/_test

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
